In [106]:
import numpy as np

In [107]:
with open('input12.txt', "r") as file:
        text = file.read().strip().split('\n')
grid = np.array([list(x) for x in text])
# Example usage:
# grid = np.array([list(x) for x in """RRRRIICCFF
# RRRRIICCCF
# VVRRRCCFFF
# VVRCCCJFFF
# VVVVCJJCFE
# VVIVCCJJEE
# VVIIICJJEE
# MIIIIIJJEE
# MIIISIJEEE
# MMMISSJEEE""".strip().split('\n')])

In [108]:
from collections import deque

def is_valid_move(x, y, grid, visited, start_value):
    rows, cols = len(grid), len(grid[0])
    return 0 <= x < rows and 0 <= y < cols and grid[x][y] == start_value and not visited[x][y]

def bfs(grid, start):
    rows, cols = len(grid), len(grid[0])
    visited = [[False for _ in range(cols)] for _ in range(rows)]
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # right, down, left, up

    queue = deque([(start, [start])])
    visited[start[0]][start[1]] = True
    start_value = grid[start]
    path2 = [start]
    while queue:
        (x, y), path = queue.popleft()

        for dx, dy in directions:
            nx, ny = x + dx, y + dy
            if is_valid_move(nx, ny, grid, visited, start_value):
                path2.append((nx, ny))
                visited[nx][ny] = True
                queue.append(((nx, ny), path + [(nx, ny)]))

    return path2

start = (0, 0)

visited = []
areas_dict = {}
counter = 0
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        start = (i, j)
        if start not in visited:
            path = bfs(grid, start)
            areas_dict[counter] = path
            visited.extend(path)
            counter += 1

In [109]:
cost = 0
for area in areas_dict.values():
    perim = 0
    for coord in area:
        i, j = coord
        for x, y in ((i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)):
            if (x, y) not in area:
                perim += 1

    cost += len(area) * perim
#    print(len(area), perim, cost)
cost

1374934